#### Download and Extract the Dataset

In [15]:
#! wget https://www.dropbox.com/s/rbajpdlh7efkdo1/male_female_face_images.zip

In [16]:
#! unzip male_female_face_images.zip

#### Import the Relevant Packages

In [17]:
from torch_snippets import *
from torchvision import transforms as T
from glob import glob
from PIL import Image
import torchvision
import torch.nn as nn
import torchvision.utils as vutils
import cv2 , numpy as np , pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#### Prepare the Dataset and DataLoader

In [4]:
#crop the images to contain only the face
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
! mkdir cropped_faces_females
! mkdir cropped_faces_males

In [22]:
def crop_images(in_folder, out_folder):
    folder_images = glob(in_folder + '/*.jpg')
    for i in range(len(folder_images)):
        original_image = cv2.imread(folder_images[i], 1)
        gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cropped_image = original_image[y:(y+h),x:(x+w),:]
            cv2.imwrite(out_folder + '/'+str(i)+'.jpg', cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR))

In [23]:
crop_images('females', 'cropped_faces_females')

In [24]:
crop_images ('males', 'cropped_faces_males')

In [14]:
transform=transforms.Compose([
                                transforms.Resize(64),
                                transforms.CenterCrop(64),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

#### Define the Faces Dataset Class 

In [25]:
class Faces(Dataset):
    def __init__(self,folders):
        super().__init__()
        self.folderfemale = folders[0]
        self.foldermale = folders[1]
        self.images = sorted(glob(self.folderfemale)) + sorted(glob(self.foldermale))
    def __len__(self):
        return len(self.images)
    def __getitem__(self,idx):
        image_path = self.images[ix]
        image = Image.open(image_path)
        image = transform(image)
        gender = np.where('female' in image_path,1,0)
        return image, torch.tensor(gender).long()